## Problem 1

The liquid-vapor equilibrium equation with two parameters $A_{12}$ and $A_{21}$:

$
\begin{aligned}
P = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)P_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)P_{1,4 dioxane}^{sat}
\end{aligned}
$

The saturation pressures, $p^{sat}$ are given by the Antoine equation:

$
\log_{10}(P^{sat}) = a_1 - \frac{a_2}{T + a_3},
$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|     &nbsp;  | $a_1$   | $a_2$    | $a_3$   |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$, so $x_2=1-x_1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table:

### First step, formulating the least square problem.

Minimize the total error between expected result and experimental result:

$\underset{A_{12},A_{21}}{\text{min}}\sum^{11}_{i=1}(P(x_{1i},A_{12},A_{21})-P_i)^2$

Because the independent value is an exponent in an exponential function, the P function is nonlinear with respect to parameters $A_{12}$ and $A_{21}$.

Apply gradient descent:

In [6]:
%reset
import torch as t
from torch.autograd import Variable
import numpy as np

T = 20 # temperature in C
x1 = t.from_numpy(np.linspace(0,1,11))
x2 = 1 - x1
Pa = t.from_numpy(np.array([28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5])) # experimental result
Psw = 10**(8.07131 - 1730.63/(T+233.426)) # for water
Pso = 10**(7.43155 - 1554.679/(T+240.337)) # for 1,4 dioxane

A = Variable(t.tensor([1.0, 2.0]), requires_grad=True)

a=0.001

# Start gradient descent
for i in range(1000):  # TODO: change the termination criterion
    loss = t.norm((x1*t.exp(A[0]*((A[1]*x2)/(A[0]*x1+A[1]*x2))**2)*Psw + x2*t.exp(A[1]*((A[0]*x1)/(A[0]*x1+A[1]*x2))**2)*Pso - Pa)**2)
    loss.backward()
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        A -= a * A.grad

        # need to clear the gradient at every step, or otherwise it will accumulate...
        A.grad.zero_()

print(A.data.numpy())
print(loss.data.numpy())

[1.9542572 1.6984679]
0.5309088658728502


3. Compare your optimized model with the data. Does your model fit well with the data?
